Importing the required libraries


In [43]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import datetime as dt
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import datetime
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
plt.style.use('seaborn-darkgrid')
  
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

Loading the dataset


In [2]:
df1 = pd.read_csv('stocknet_trn_data.csv')
df2 = pd.read_csv('stocknet_trn_data_targets.csv', names = ['label', 'close'])
df3 = pd.read_csv('stocknet_tst_data.csv')

In [3]:
df1.head()

,Open,High,Low,Volume
0,75.430000,75.910004,75.139999,3436700.0
1,124.930000,124.940002,124.169998,2366700.0
2,84.339996,84.370003,82.370003,11338700.0
3,56.770000,57.150002,56.529999,413300.0
4,47.419998,48.090000,46.759998,9531100.0


In [4]:
df2.head()

,label,close
0,d1,75.650002
1,d2,124.449997
2,d3,82.660004
3,d4,57.009998
4,d5,47.860001


In [5]:
df3.head()

,Open,High,Low,Volume
0,26.299999,26.520000,25.959999,32642100
1,95.550003,95.739998,94.019997,1826300
2,54.770000,55.360001,54.590000,5656800
3,125.589996,125.820000,123.660004,4151600
4,20.709999,20.809999,20.670000,10489800


In [6]:
df1[df1.isna().any(axis=1)]

,Open,High,Low,Volume
7642,NaN,NaN,NaN,NaN
46002,NaN,NaN,NaN,NaN
61502,NaN,NaN,NaN,NaN


In [7]:
df2[df2.isna().any(axis=1)]

,label,close
7642,d7643,NaN
46002,d46003,NaN
61502,d61503,NaN


In [8]:
df3[df3.isna().any(axis=1)]

,Open,High,Low,Volume


In [9]:
df1.dropna(how='any', inplace=True)

In [10]:
df2.dropna(how='any', inplace=True)

In [11]:
df1.isna().sum()

Open      0
High      0
Low       0
Volume    0
dtype: int64

In [12]:
df2.isna().sum()

label    0
close    0
dtype: int64

In [13]:
df1.head()

,Open,High,Low,Volume
0,75.430000,75.910004,75.139999,3436700.0
1,124.930000,124.940002,124.169998,2366700.0
2,84.339996,84.370003,82.370003,11338700.0
3,56.770000,57.150002,56.529999,413300.0
4,47.419998,48.090000,46.759998,9531100.0


In [14]:
df2.head()

,label,close
0,d1,75.650002
1,d2,124.449997
2,d3,82.660004
3,d4,57.009998
4,d5,47.860001


In [15]:
df = pd.concat([df1, df2], axis=1, join='inner')

In [16]:
df.head()

,Open,High,Low,Volume,label,close
0,75.430000,75.910004,75.139999,3436700.0,d1,75.650002
1,124.930000,124.940002,124.169998,2366700.0,d2,124.449997
2,84.339996,84.370003,82.370003,11338700.0,d3,82.660004
3,56.770000,57.150002,56.529999,413300.0,d4,57.009998
4,47.419998,48.090000,46.759998,9531100.0,d5,47.860001


In [17]:
df.isna().sum()

Open      0
High      0
Low       0
Volume    0
label     0
close     0
dtype: int64

In [45]:
df_copy = df.copy(deep=True)

In [46]:
# Moving averages

def computeSMA(data, window):
    # simple moving average
    sma = data.rolling(window=window).mean()

    return sma

def computeEMA(data, span):
    # simple moving average
    ema = data.ewm(span=span, adjust=False).mean()

    return ema

def construct_df(df_input):
    
    # compute both types of moving averages
    for i in range(50, 250, 50):
        
        df_input['SMA_{}'.format(i)] = computeSMA(df_input['close'], i)
    for i in range(50, 250, 50):
       
        df_input['EMA_{}'.format(i)] = computeEMA(df_input['close'], i)
    
    return df_input  

In [47]:
df_moving_avg = construct_df(df_copy)

In [18]:
df_lstm=df.reset_index()['close']

In [48]:
df_moving_avg.tail()

,Open,High,Low,Volume,label,close,SMA_50,SMA_100,SMA_150,SMA_200,EMA_50,EMA_100,EMA_150,EMA_200
97727,103.650002,103.809998,103.089996,3678200.0,d97728,103.230003,69.671267,6150.984391,5187.895920,3930.826365,1383.506084,3172.174205,3835.428156,4054.154229
97728,64.419998,64.730003,64.190002,20273100.0,d97729,64.620003,69.869467,6145.969285,5187.793368,3930.611015,1331.785061,3110.638478,3785.483678,4014.457371
97729,581.070007,581.400024,571.070007,3416500.0,d97730,575.140015,80.915967,6150.668785,5190.734168,3933.144415,1302.112707,3060.430587,3742.962569,3980.235308
97730,120.650002,120.650002,114.959999,13261700.0,d97731,115.400002,82.120567,6151.226185,5191.340101,3928.248915,1255.574954,3002.113150,3694.915383,3941.779235
97731,64.870003,64.959999,64.309998,667000.0,d97732,64.720001,81.560367,6151.319185,5191.347901,3928.462215,1208.874759,2943.946949,3646.833325,3903.201531


In [49]:
X_avg = df_moving_avg[['Open', 'High', 'Low', 'Volume', 'SMA_50', 'SMA_100',
       'SMA_150', 'SMA_200', 'EMA_50', 'EMA_100', 'EMA_150', 'EMA_200']]

In [50]:
X_avg.dropna(how='any', inplace=True)

In [51]:
y_avg = df_moving_avg.iloc[:, 5:6]

In [52]:
y_avg.dropna(how='any', inplace=True)

In [64]:
y_avg = np.resize(y_avg, (97530, 1))

In [65]:
y_avg.shape

(97530, 1)

In [19]:
df_lstm.head()

0     75.650002
1    124.449997
2     82.660004
3     57.009998
4     47.860001
Name: close, dtype: float64

In [20]:
scaler = MinMaxScaler(feature_range=(0,1))
df_lstm1=scaler.fit_transform(np.array(df_lstm).reshape(-1,1))

In [21]:
training_size=int(len(df_lstm1)*0.65)
test_size=len(df_lstm1)-training_size
train_data,test_data=df_lstm1[0:training_size,:],df_lstm1[training_size:len(df_lstm1),:1]

In [22]:
training_size,test_size

(63523, 34206)

In [23]:
train_data

array([[2.71637303e-04],
       [4.51671239e-04],
       [2.97498745e-04],
       ...,
       [2.64295762e-04],
       [2.41643934e-05],
       [3.18084573e-04]])

In [24]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [25]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [26]:
print(X_train.shape), print(y_train.shape)

(63422, 100)
(63422,)


(None, None)

In [27]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [30]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=10,batch_size=64,verbose=1)

Epoch 1/10
991/991 [==============================] - 83s 83ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 2/10
991/991 [==============================] - 81s 82ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 3/10
991/991 [==============================] - 82s 83ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 4/10
991/991 [==============================] - 80s 81ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 5/10
991/991 [==============================] - 80s 81ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 6/10
991/991 [==============================] - 80s 80ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 7/10
991/991 [==============================] - 79s 80ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 8/10
991/991 [==============================] - 80s 81ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 9/10
991/991 [==============================] - 80s 81ms/step - loss: 0.0062 - val_loss: 0.0069
Epoch 10/10
991/991 [==============================] - 79s 80ms/step - loss: 0.006

In [35]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [36]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [39]:
test_predict

array([[1463.6487],
       [1463.6493],
       [1463.6498],
       ...,
       [1463.6309],
       [1463.6315],
       [1463.6393]], dtype=float32)

In [37]:
#RMSE

import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

1464.7286832801778

In [38]:
math.sqrt(mean_squared_error(ytest,test_predict))

1464.865762386113

In [41]:
#MBE

def MBE(y_true, y_pred):
    '''
    Parameters:
        y_true (array): Array of observed values
        y_pred (array): Array of prediction values

    Returns:
        mbe (float): Biais score
    '''
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true = y_true.reshape(len(y_true),1)
    y_pred = y_pred.reshape(len(y_pred),1)   
    diff = (y_true-y_pred)
    mbe = diff.mean()
    print('MBE = ', mbe)

In [42]:
MBE(ytest,test_predict)

MBE =  -1464.8611556259025


In [44]:
#MAPE

mean_absolute_percentage_error(ytest,test_predict)

193278034309746.28

Now using moving average as a feature

In [68]:
df_lstm_avg_50 = X_avg['SMA_50']

In [69]:
df_lstm_sma=scaler.fit_transform(np.array(df_lstm_avg_50).reshape(-1,1))

In [70]:
training_size=int(len(df_lstm_sma)*0.65)
test_size=len(df_lstm_sma)-training_size
train_data_avg,test_data_avg=df_lstm_sma[0:training_size,:],df_lstm_sma[training_size:len(df_lstm_sma),:1]

In [71]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train_avg, y_train_avg = create_dataset(train_data_avg, time_step)
X_test_avg, ytest_avg = create_dataset(test_data_avg, time_step)

	

In [73]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train_avg =X_train_avg.reshape(X_train_avg.shape[0],X_train_avg.shape[1] , 1)
X_test_avg = X_test_avg.reshape(X_test_avg.shape[0],X_test_avg.shape[1] , 1)

In [74]:
model.fit(X_train_avg,y_train_avg,validation_data=(X_test_avg,ytest_avg),epochs=10,batch_size=64,verbose=1)

Epoch 1/10
989/989 [==============================] - 81s 82ms/step - loss: 0.0018 - val_loss: 8.5020e-04
Epoch 2/10
989/989 [==============================] - 81s 82ms/step - loss: 7.4568e-04 - val_loss: 8.1307e-04
Epoch 3/10
989/989 [==============================] - 80s 81ms/step - loss: 7.3895e-04 - val_loss: 8.0114e-04
Epoch 4/10
989/989 [==============================] - 80s 81ms/step - loss: 7.3237e-04 - val_loss: 8.0772e-04
Epoch 5/10
989/989 [==============================] - 81s 81ms/step - loss: 7.2970e-04 - val_loss: 8.1190e-04
Epoch 6/10
989/989 [==============================] - 80s 81ms/step - loss: 7.2344e-04 - val_loss: 7.9867e-04
Epoch 7/10
989/989 [==============================] - 81s 82ms/step - loss: 7.2312e-04 - val_loss: 8.1340e-04
Epoch 8/10
989/989 [==============================] - 80s 81ms/step - loss: 7.0957e-04 - val_loss: 7.7334e-04
Epoch 9/10
989/989 [==============================] - 80s 81ms/step - loss: 7.0631e-04 - val_loss: 7.8001e-04
Epoch 10/10
98

In [76]:
test_predict=model.predict(X_test_avg)

In [77]:
MBE(ytest_avg, test_predict) #MBE

MBE =  -0.0018691890205891008


In [78]:
mean_absolute_percentage_error(ytest_avg, test_predict) #MAPE

1.2462094050629229

In [79]:
math.sqrt(mean_squared_error(ytest_avg, test_predict)) #RMSE

0.028404206388774632